In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
game_reviews_df = pd.read_csv('steam_reviews.csv')
# print(game_reviews_df)

clean_game_reviews_df = game_reviews_df.drop(['review', 'is_early_access_review'], axis=1)
# print(clean_game_reviews_df)

matrix = clean_game_reviews_df.pivot_table(columns='title', index='date_posted', values='recommendation')
print(matrix)

Empty DataFrame
Columns: []
Index: [2010-12-20, 2011-02-05, 2011-05-31, 2011-06-05, 2011-06-15, 2011-07-02, 2011-07-03, 2011-11-27, 2011-12-21, 2011-12-29, 2012-01-06, 2012-01-19, 2012-02-17, 2012-07-12, 2012-07-13, 2012-07-14, 2012-07-19, 2012-07-30, 2012-08-15, 2012-10-07, 2012-12-18, 2013-01-03, 2013-01-26, 2013-02-15, 2013-03-02, 2013-03-24, 2013-03-27, 2013-03-30, 2013-04-07, 2013-05-05, 2013-05-09, 2013-05-30, 2013-06-08, 2013-06-21, 2013-07-12, 2013-07-15, 2013-07-24, 2013-07-26, 2013-07-30, 2013-08-01, 2013-08-08, 2013-08-13, 2013-08-18, 2013-09-15, 2013-09-20, 2013-09-21, 2013-10-02, 2013-10-04, 2013-10-06, 2013-11-15, 2013-11-25, 2013-11-29, 2013-11-30, 2013-12-01, 2013-12-02, 2013-12-03, 2013-12-04, 2013-12-05, 2013-12-06, 2013-12-07, 2013-12-10, 2013-12-11, 2013-12-12, 2013-12-13, 2013-12-14, 2013-12-15, 2013-12-16, 2013-12-17, 2013-12-18, 2013-12-19, 2013-12-20, 2013-12-21, 2013-12-22, 2013-12-23, 2013-12-24, 2013-12-25, 2013-12-26, 2013-12-27, 2013-12-28, 2013-12-29, 2013

/tmp/ipykernel_1443/2373340343.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  matrix = clean_game_reviews_df.pivot_table(columns='title', index='date_posted', values='recommendation')
